### In this notebook, you will:
1. Get "live" data from all the visible PVTA buses
2. Store that data into a SQLite or Parquet file
3. Query that data using one of:
    1. Raw SQL
    2. A Python ORM called `Peewee`
    3. Panda Dataframe operations

## Start with a request to the API:

In [53]:
import requests
response = requests.get("https://bustracker.pvta.com/InfoPoint/rest/routes/getvisibleroutes")

### Now we extract the list of vehicles:

In [54]:
routes = (rt for rt in response.json())
vehicles = [v for rt in routes for v in rt['Vehicles']]

### It's useful to convert the `LastUpdated` field into this format:

In [55]:
import re
import datetime
for v in vehicles:
    timestamp_ms = re.search(r'\d+', v['LastUpdated']).group()[:]
    v['LastUpdated_timestamp_ms'] = timestamp_ms
# Printing just one vehicle's data, see the peewee model below for more info on each field:
vehicles[:1]

[{'BlockFareboxId': 65,
  'CommStatus': 'GOOD',
  'Destination': 'Chicopee Big Y',
  'Deviation': 0,
  'Direction': 'N',
  'DirectionLong': 'Northbound',
  'DisplayStatus': 'On Time',
  'StopId': 6351,
  'CurrentStatus': None,
  'DriverName': None,
  'DriverLastName': None,
  'DriverFirstName': None,
  'DriverFareboxId': 0,
  'VehicleFareboxId': 1653,
  'GPSStatus': 2,
  'Heading': 92,
  'LastStop': 'Front / Cyman',
  'LastUpdated': '/Date(1712866260000-0400)/',
  'Latitude': 42.148017,
  'Longitude': -72.593896,
  'Name': '1653',
  'OccupancyStatus': 1,
  'OnBoard': 6,
  'OpStatus': 'ONTIME',
  'RouteId': 20001,
  'RunId': 1557285,
  'Speed': None,
  'TripId': 1555,
  'VehicleId': 1653,
  'SeatingCapacity': 40,
  'TotalCapacity': 50,
  'PropertyName': 'SATCO',
  'OccupancyStatusReportLabel': 'Many Seats Available',
  'LastUpdated_timestamp_ms': '1712866260000'}]

### Approach 1: SQLite with Peewee

In [56]:
# First, let's create a DB file if it does not exist already:
with open('pvta-sqlite-data.sqlite', 'a+') as f:
    pass

In [57]:
# Now, create the peewee sqlite model:
import json, re, peewee
import datetime
from peewee import Model, CharField, DoubleField, IntegerField, DateTimeField

db = peewee.SqliteDatabase("pvta-sqlite-data.sqlite")

class VehiclePositionRecord(Model):
    # Different values, but both appear to ID a vehicle:
    Name =  CharField()
    VehicleId = IntegerField()
    # Position info:
    Latitude = DoubleField()
    Longitude = DoubleField()
    # Time info:
    LastUpdated = DateTimeField()
    # Ex: 10043 for the B43
    RouteId = IntegerField()
    # Lateness in minutes. (Could be more precise by retreiving the schedule data, but still useful)
    Deviation = IntegerField()
    # NOT UNIQUE; RunId seems to update everytime the vehicle starts running for the day (or leaves the garage?)
    RunId = IntegerField()
    # NOT UNIQUE; updated everytime the vehicle starts moving from the 1st stop in a route. The value is simply the scheduled time (7am -> 700)
    TripId = IntegerField()
    # Other useful info
    Direction =  CharField()
    Destination = CharField()
    LastStop = CharField(null = True)

    # Not too important, a lot of these are NULL:
    OpStatus = CharField(null = True)
    Heading = IntegerField(null = True)
    OnBoard = IntegerField(null = True)
    TotalCapacity = IntegerField(null = True)
    BlockFareboxId = IntegerField(null = True)
    CommStatus =  CharField(null = True)
    OccupancyStatus = IntegerField(null = True)
    DirectionLong =  CharField(null = True)
    DisplayStatus =  CharField(null = True)
    DriverFareboxId = IntegerField(null = True)
    VehicleFareboxId = IntegerField(null = True)
    GPSStatus = IntegerField(null = True)
    SeatingCapacity = IntegerField(null = True)
    PropertyName =  CharField(null = True)
    OccupancyStatusReportLabel =  CharField(null = True)
    StopId = IntegerField(null = True)

    class Meta:
        database = db
        # This prevents having two records of the same vehicle at the same time
        indexes = (
          (('LastUpdated', 'VehicleId'), True),
        )

In [58]:
# Needed for first run, doesn't matter if you run this or not afterwards. Initialize tables:
db.create_tables([VehiclePositionRecord], safe = True)


#### Saving the vehicle records to the database:

In [ ]:
# Add the vehicles:
for v in vehicles:
    vehicle_record = VehiclePositionRecord(
            Name=v['Name'],
            VehicleId=v['VehicleId'],
            Latitude=v['Latitude'],
            Longitude=v['Longitude'],
            # the '-3' removes the last 3 digits, converting milliseconds->seconds
            LastUpdated=datetime.datetime.fromtimestamp(int(re.search(r'\d+', v['LastUpdated_timestamp_ms']).group()[:-3])),
            RouteId=v['RouteId'],
            Direction=v['Direction'],
            Destination=v['Destination'],
            Deviation=v['Deviation'],
            RunId=v['RunId'],
            TripId=v['TripId'],
            LastStop=v['LastStop'],
            OpStatus=v['OpStatus'],
            Heading=v['Heading'],
            OnBoard=v['OnBoard'],
            TotalCapacity=v['TotalCapacity'],
            BlockFareboxId=v['BlockFareboxId'],
            CommStatus=v['CommStatus'],
            OccupancyStatus=v['OccupancyStatus'],
            DirectionLong=v['DirectionLong'],
            DisplayStatus=v['DisplayStatus'],
            DriverFareboxId=v['DriverFareboxId'],
            VehicleFareboxId=v['VehicleFareboxId'],
            GPSStatus=v['GPSStatus'],
            SeatingCapacity=v['SeatingCapacity'],
            PropertyName=v['PropertyName'],
            OccupancyStatusReportLabel=v['OccupancyStatusReportLabel'],
            StopId=v['StopId']
        )
    vehicle_record.save()

#### Let's select B43 buses, going west, since 9pm April 10th:

In [36]:
start_date = datetime.datetime(2024, 4, 10, 21)
end_date = datetime.datetime.now()

s = VehiclePositionRecord.select().where(VehiclePositionRecord.RouteId == 10043)
# Constraints can be chained separately or together using '&' (AND) or '|' (OR)
s = s.where(
    (VehiclePositionRecord.Direction == 'W') & 
    (VehiclePositionRecord.LastUpdated.between(start_date, end_date))
)

for vRecord in s:
    print()
    print(vRecord.LastUpdated, vRecord.RunId, vRecord.VehicleId, vRecord.LastStop, vRecord.Direction)
    print()


2024-04-10 21:31:27 1545622 701 Prospect Street W


2024-04-10 21:31:28 1545613 502 Haigis Mall W


2024-04-10 21:46:43 1545622 701 Prospect Street W


2024-04-10 21:48:19 1545613 502 Russell/Russell (The Stables) W


2024-04-11 16:01:27 1545616 701 Russell/Rte 9 (Holiday Inn Express) W


2024-04-11 16:01:29 1545610 412 Fearing Street (In) W



#### Let's show the percentage of B43 buses over 5 minutes late:

In [52]:
five_mins_late = VehiclePositionRecord.select().where(VehiclePositionRecord.RouteId == 10043).where(VehiclePositionRecord.Deviation > 5)
all_b43 = VehiclePositionRecord.select().where(VehiclePositionRecord.RouteId == 10043)
# Could be 0 if you don't have enough data, it was around 18% on my copy with ~170k records.
(five_mins_late.count() / all_b43.count()) * 100

9.090909090909092

Since all the data is in a `sqlite` file, you can also use sqlite to query the data

### Approach 2: Pandas

In [38]:
# First, create a dataframe from the vehicles list:
import pandas as pd
import json
df = pd.read_json(json.dumps(vehicles))

# Add a datetime column for easier querying of the `LastUpdated` field:
df['LastUpdated_datetime'] = pd.to_datetime(df['LastUpdated_timestamp_ms'], unit='ms')

In [39]:
# Let's show rows from the B43:
df[df['RouteId'] == 10043]

,BlockFareboxId,CommStatus,Destination,Deviation,Direction,DirectionLong,DisplayStatus,StopId,CurrentStatus,DriverName,...,RunId,Speed,TripId,VehicleId,SeatingCapacity,TotalCapacity,PropertyName,OccupancyStatusReportLabel,LastUpdated_timestamp_ms,LastUpdated_datetime
82,4308,GOOD,Amherst College via UMass,1,E,East,On Time,0,NaN,NaN,...,1545622,NaN,1540,411,40,56,VATCO,Many Seats Available,1712865684000,2024-04-11 20:01:24
83,4303,GOOD,Amherst College via UMass,0,E,East,On Time,0,NaN,NaN,...,1545608,NaN,1520,501,49,69,VATCO,Many Seats Available,1712865685000,2024-04-11 20:01:25
84,4304,GOOD,Northampton via Hampshire Mall,6,W,West,Late,0,NaN,NaN,...,1545616,NaN,1530,701,40,56,VATCO,Many Seats Available,1712865687000,2024-04-11 20:01:27
85,4309,GOOD,Northampton via Hampshire Mall,0,W,West,On Time,0,NaN,NaN,...,1545610,NaN,1550,412,40,56,VATCO,Many Seats Available,1712865689000,2024-04-11 20:01:29
86,4305,GOOD,Amherst College via UMass,1,E,East,On Time,0,NaN,NaN,...,1545620,NaN,1600,419,40,56,VATCO,Many Seats Available,1712865696000,2024-04-11 20:01:36


#### Append the pandas data to a parquet file:

In [43]:
# Creates file if not already existing:
try:
    old_df = pd.read_parquet('pvta-parquet-data.parquet')
    df = pd.concat([df, old_df])
except FileNotFoundError:
    pass
df.to_parquet('pvta-parquet-data.parquet')

In [ ]:
#TODO: select buses in a particular time frame.